In [ ]:
%load_ext autoreload
%autoreload 2

import sys

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'] if 'google.colab' in sys.modules else [])
#proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

import os
os.environ["MOSEKLM_LICENSE_FILE"] = os.getenv("HOME") + "/mosek/mosek.lic"

In [ ]:
# This first cell has some updates/additions to the ConvexSet class hierarchy
# (which I aim to push in some form to drake).

import numpy as np
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
)

# Compute the "center" of a HPolyhedron.
ipopt = IpoptSolver()
def center(self):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(self.ambient_dimension())
    r = prog.NewContinuousVariables(1)[0]

    A_row_norm = np.linalg.norm(self.A(), axis=1)
    lhs = self.A().dot(x) + A_row_norm * r
    prog.AddLinearConstraint(le(lhs, self.b()))
    prog.AddLinearConstraint(r >= 0)
    prog.AddLinearCost(-r)

    result = ipopt.Solve(prog)
    assert result.is_success()

    return result.GetSolution(x)

HPolyhedron.center = center

import meshcat.geometry as g
from pydrake.all import RotationMatrix, RigidTransform
from scipy.spatial import ConvexHull, HalfspaceIntersection

def set_meshcat_object(meshcat, set, color=0xdd9999, wireframe=False):
    material = g.MeshLambertMaterial(color=color, wireframe=wireframe)

    if isinstance(set, np.ndarray):  # Render a point as a sphere.
        meshcat.set_transform(RigidTransform(set).GetAsMatrix4())
        meshcat.set_object(g.Sphere(.01), material)
    elif isinstance(set, HPolyhedron):
        assert set.ambient_dimension() == 3
        halfspaces = np.column_stack((set.A(), -set.b()))
        P = HalfspaceIntersection(halfspaces, set.center())
        vertices = P.intersections
        hull = ConvexHull(vertices)  # orders vertices counterclockwise
        vertices = vertices[hull.vertices]
        faces = hull.simplices
        meshcat.set_object(g.TriangularMeshGeometry(vertices, faces), material)
    elif isinstance(set, Hyperellipsoid):
        shape, X_WG = set.ToShapeWithPose()
        meshcat.set_transform(X_WG.GetAsMatrix4())
        meshcat.set_object(g.Ellipsoid([shape.a(), shape.b(), shape.c()]), material)
    else:
        print(type(set))
        raise NotImplementedError



In [ ]:
# This cell builds a default environment for testing, with the planar kuka
# reaching into a shelf.  It's very nice, because it has a 3D configuration
# space, which I can visualize without projections.

from meshcat import Visualizer
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser,
    MathematicalProgram, Solve,
    ConnectMeshcatVisualizer,
    Role, Sphere
)
from manipulation.scenarios import AddShape, AddPlanarIiwa, AddWsg
from manipulation.utils import FindResource
from IPython.display import display

def make_environment(robot=True, gripper=True, goal=True):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    if robot:
        iiwa = AddPlanarIiwa(plant, simple_collision=True)
        if gripper:
            wsg = AddWsg(plant, iiwa, roll=0.0, welded=True)
    if goal:
        sphere = AddShape(plant, Sphere(0.02), "sphere", collision=False)
        X_WO = RigidTransform([0.6, 0, 0.65])
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("sphere"), X_WO)

    parser = Parser(plant)
    shelves = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))

    plant.Finalize()

    visualizer = ConnectMeshcatVisualizer(
        builder,
        scene_graph,
        zmq_url=zmq_url,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    visualizer.load()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

    return diagram, plant, scene_graph

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())
make_environment();

In [ ]:
# This cell implements/tests IRIS in 3D (ignoring the robot geometry).
# This is a very small scale problem, but it works really well!

from meshcat import Visualizer

vis.delete()

from pydrake.all import Iris, IrisOptions, MakeIrisObstacles, Variable

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

diagram, plant, scene_graph = make_environment(robot=False)
domain = HPolyhedron.MakeBox([0, -.5, 0], [1, .5, 1])
context = diagram.CreateDefaultContext()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
xs = np.array([[0.3, 0, 0.45],
               [0.5, 0, 0.85],
               [0.5, 0, 0.65],
               [0.5, 0, 0.45],
               [0.5, 0, 0.2],
               [0.5, 0.45, 0.45],
               [0.5, -0.45, 0.45]])
obstacles = MakeIrisObstacles(query)
options = IrisOptions()
options.require_sample_point_is_contained = True
x = [Variable('x'), Variable('y'), Variable('z')]
for i in range(xs.shape[0]):
    region = Iris(obstacles, xs[i,:], domain, options)
    set_meshcat_object(vis[f"iris_region_{i}"], region, wireframe=True)

In [ ]:
# Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    DrealSolver, eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

# Maybe this one doesn't actually deserve to be part of the main class,
# or it needs to be renamed.
# It's really the gradient of solution to the GrowthVolume optimization.
def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane

diagram, plant, scene_graph = make_environment(robot=True, gripper=False)
lb = np.array([0, -.5, 0])
ub = np.array([1, .5, 1])
context = diagram.CreateDefaultContext()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()
dReal = DrealSolver()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()

vis.delete()

def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)

dReal_polytope_tol = .1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
    return result.is_success(), result.GetSolution(q)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b, guess=None):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")

    if guess is not None:
        prog.SetInitialGuess(q, guess)

    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)

    # TODO: Remove these.  They're for debugging only.
    #set_meshcat_object(vis['setA'],setA)
    #set_meshcat_object(vis['setB'],setB)

    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        # TODO: Update this to use MBP<double> and Jacobians if I'm going to keep it.
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    vis = Visualizer(zmq_url=zmq_url)
    set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
    assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(lb, ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = True
    use_dReal = True
    ad_bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    if use_autodiff:
        bodies = ad_bodies

    iteration = 0
    while True:
        ## Find separating hyperplanes

        num_faces = 2*len(lb)
        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                success, growth, qstar = GrowthVolume(E,
                    bodies[geomA], bodies[geomB],
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if use_dReal:
                tries = 0
                while True:
                    reachable, counter_example = CheckVolume(
                        E, sym_bodies[geomA], sym_bodies[geomB], sets[geomA], sets[geomB],
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, np.inf)
                    if not reachable:
                        print("unreachable")
                        break
                    else:
                        z = E.A() @ (counter_example - E.center())
                        dreal_growth = z.dot(z)
                        print(f"counter_example = {counter_example}, growth = {dreal_growth}")
                        success, growth, qstar = GrowthVolume(E,
                            bodies[geomA], bodies[geomB],
                            sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, counter_example)
                        if success:
                            print(f"snopt_example={qstar}, growth = {growth}")
                            # Add a face to the polytope
                            A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                            num_faces += 1
                        if np.all(A[:num_faces,:] @ counter_example <= b[:num_faces] - dReal_polytope_tol):
                            # Then also add the counter-example
                            A[num_faces,:], b[num_faces] = E.TangentPlane(counter_example)
                            num_faces += 1
                    tries += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
        set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.MaximumVolumeInscribedEllipsoid()
        set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


q = np.array([0.55, 0, 0.65])
#q0 = plant.GetPositions(plant.GetMyContextFromRoot(context))
#iris_cspace(query, q, require_containment_points=[q], iteration_limit=100);


In [ ]:
from pydrake.all import InverseKinematics

# Let's find IRIS c-space regions, by calling IK on the 3D samples.

diagram, plant, scene_graph = make_environment(robot=True, gripper=False)
context = diagram.CreateDefaultContext()
vis.delete()

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

xs = np.array([[0.3, 0, 0.45],
               [0.5, 0, 0.85],
               [0.55, 0, 0.65]])
regions = []
hand_body = plant.GetBodyByName("iiwa_link_ee_kuka")
hand_frame = hand_body.body_frame()
plant_context = plant.GetMyContextFromRoot(context)
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()

q0 = plant.GetPositions(plant_context)
qs = np.zeros(xs.shape)
for i in range(xs.shape[0]):
    ik = InverseKinematics(plant, plant_context)
    ik.AddMinimumDistanceConstraint(0.001, 0.01)
    ik.AddPositionConstraint(hand_frame, [0, 0, 0], plant.world_frame(), xs[i,:], xs[i,:])
    ik.prog().SetInitialGuess(ik.q(), q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print(xs[i,:])
        assert result.is_success()
    qs[i,:] = result.GetSolution(ik.q())

for i in range(qs.shape[0]):
    regions.append(iris_cspace(query, qs[i,:], require_containment_points=[qs[i,:]], iteration_limit=100))


In [ ]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra

spp = BsplineTrajectoryThroughUnionOfHPolyhedra(qs[1,:], qs[2,:], regions)
spp.set_max_velocity([.4, .4, .4])
print(spp.num_regions())
traj = spp.Solve()
print(traj.start_time())
print(traj.end_time())

for q in traj.control_points():
    if not any([r.PointInSet(q) for r in regions]):
        print(f"control point {q} in not in any region")

from pydrake.all import MultibodyPlant, SceneGraph, MultibodyPositionToGeometryPose, TrajectorySource, Simulator

def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    iiwa = AddPlanarIiwa(plant, simple_collision=True)

    parser = Parser(plant)
    shelves = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    diagram = builder.Build()

    simulator = Simulator(diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())

vis.delete()
visualize_trajectory(traj)

In [ ]:
# This never returned last time I ran it....

from pydrake.all import ShortestPathProblem, L2NormCost, Binding_L2NormCost, Point

# Now let's formulate a SPP.
l2norm = L2NormCost(np.hstack((np.eye(3), -np.eye(3))), [0, 0, 0])
vertices = []
def shortest_path(q_start, q_goal):
    spp = ShortestPathProblem()
    for r in regions:
        vertices.append(spp.AddVertex(r))
    for i in range(len(regions)):
        for j in range(len(regions)):
            prog = MathematicalProgram()
            q = prog.NewContinuousVariables(3)
            regions[i].AddPointInSetConstraints(prog, q)
            regions[j].AddPointInSetConstraints(prog, q)
            result = Solve(prog)
            if result.is_success():
                # Then I'm in the intersection
                e = spp.AddEdge(i, j)
                e.AddCost(Binding_L2NormCost(l2norm, np.hstack((e.xu(), e.xv()))))
    source = spp.AddVertex(Point(q_start))
    spp.set_source(source)
    target = spp.AddVertex(Point(q_goal))
    spp.set_target(target)
    for i in range(len(regions)):
        if regions[i].PointInSet(q_start):
            e = spp.AddEdge(source, vertices[i])
            e.AddCost(Binding_L2NormCost(l2norm, np.hstack((e.xu(), e.xv()))))
        if regions[i].PointInSet(q_goal):
            e = spp.AddEdge(vertices[i], target)
            e.AddCost(Binding_L2NormCost(l2norm, np.hstack((e.xu(), e.xv()))))
    print("Solving...")
    result = spp.Solve()

shortest_path(qs[2,:], qs[4,:])